# Домашнее задание 3

# 1. Введение.

В этом разделе нам необходимо построить прогнозные модели для курсов валют. На семинарах мы обсуждали, что прогнозировать цены котировок стандартными линейными моделями довольно бесперспективно. Так как такие данные близки к модели случайного блуждания, оптимальным прогнозом для них часто оказывается наивный. Однако если перейти к более низкой частоте (например, к месячным данным), то некоторого превосходства над наивной всё же можно добиться.

В приложенном датасете currencies.csv находятся следующие величины:
1.  Курсы ряда валют по отношению к доллару. 

    Курс EUR/USD - Евро Доллар США
    
    Курс CNY/USD - Китайский юань Доллар США
    
    Курс INR/USD - Индийская рупия Доллар США
    
    Курс JPY/USD - Японская йена Доллар США
    
    Курс GBR/USD - Британский фунт Доллар США
    
    Курс CHF/USD - Швейцарский франк Доллар США
    
    Курс BRL/USD - Бразильский реал Доллар США
    
    Курс IDR/USD - Индонезийская рупия Доллар США

2. Цена нефти BRENT за баррель и цена газа в Европе за mmbtu (Британская тепловая единица)
3. ff_rate -- ставка ФРС США

Нашей задачей будет построить прогнозную модель для всех курсов валют из пункта 1. Цены нефти, газа и ставка ФРС даны дополнительно, можете использовать их если посчитаете необходимым, но в базовой версии можно использовать только временные ряды курсов валют.

### Импорт данных

Импортируйте данные из файла. Приведите информацию о датах в один столбец типа datetime. Например, чтобы каждая точка отображала дату начала месяца. 

In [ ]:
# ༼ つ ◕_◕ ༽つ

Писать для каждого ряда одномерную модель несколько расточительно, хотя и не лишено смысла. Этот процесс необходимо автоматизировать. В простом для понимания варианте можно свести прогноз каждой валюты к табличной задаче и подобрать регрессоры из остальных рядов. Мы таким уже занимались, поэтому попробуем зайти с немного другой стороны.

Весь рынок и в частности рынок валют -- единый механизм, находящийся в некотором равновесии и иногда от него отклоняющийся. Значит, у рядов должен быть общий тренд. Формально такая ситуация называется коинтеграцией и это тема отдельной лекции, нам она будет нужна только для идеи. Более подробно про коинтеграцию можно почитать вот в этом [конспекте](https://vk.com/doc126754362_567660819?hash=AvDGHaO92KX7exjBCleLZsEGHPPX2iZfCqae2Fijkng). 

### 1. (0.5 балла) Визуализация

Давайте увидим это явно. Возьмите все курсы валют, отнормируйте их c помощью StandardScaler из sklearn и изобразите результат на одном графике. 

График будет немного шумный, но вы должны заметить что в целом валюты движутся по схожим траекториям, а некоторые группируются.

In [ ]:
# ༼ つ ◕_◕ ༽つ

Можно ли выделить эти общие тренды на рынке? Да! Можно просто рассмотреть все валюты как один многомерный вектор и понизить его размерность.

Конечно, для временных рядов существуют специфические методы понижения размерности, но они выходят за рамки нашего курса. Поэтому мы воспользуемся не слишком подходящим, но зато простым и знакомым методом главных компонент (PCA). PCA никак не учитывает временную зависимость точек, но мы позволим себе пренебречь этим.

### 2. (0.5 балла) Визуализация главных компонент

Примените PCA на рядах из восьми валют и отберите три первые главные компоненты. Изобразите их на одном графике. Сравните с предыдущим графиком. Компоненты будут примерно похожи на тренды нескольких разных групп валют.

In [ ]:
# ༼ つ ◕_◕ ༽つ

# 2. VAR для каждой валюты

Хочется построить какую-то общую модель, которая учтёт эти тренды на рынке и позволит с её помощью прогнозирвать отдельные валюты. Мы проходили только VAR(p) из прогнозирующих моделей, но её достаточно в большинстве случаев. Также мы обсуждали, что количество параметров в VAR(p)-модели равно 

$$n^2 p + n + \frac{n(n+1)}{2},$$

где n число уравнений системы. Данное выражение растёт квадратично по n. Значит, количество уравнений нам бы хотелось минимизировать. У нас в наличии 8 валют и ещё потенциально 3 дополнительных переменных. Это слишком много для стандартной VAR-модели, необходимо сократить размерность.

Будем для каждой валюты строить VAR(p) модель со следующими рядами: ряд этой валюты + из всех остальных рядов выделим PCA-компоненты. Дальнейшие шаги необходимо будет сделать для каждой из валют в списке, поэтому рекомендуется заранее писать код так, чтобы его было легко встроить в цикл. Например, разбивать код на функции и т.п. В качестве референса предлагается заполнить класс PartialForecaster. Это будет класс для прогнозирвоания одной валюты на основе всех остальных. Ваша реализация может отличаться, меняйте параметры или добавляйте свои методы в класс если необходимо.

1. (0.5 балла) Реализуйте метод fit_pca, который вычисляет главные компоненты от фичей и сохраняет их. Компоненты должны покрывать 95% дисперсии оригинальных данных. Не добавьте в вычисление PCA целевую фичу, это будет сильная утечка!

2. (0.5 балла) Реализуйте метод fit_var. В нём необходимо склеить целевую переменную и полученные главные компоненты в один датасет и обучить на этом VAR(p)-модель.

    Важное замечание. По-хорошему нужно оценивать модель на стационарных данных. Если бы мы оценивали модель методом максимального правдоподобия с ограничениями на стационарность, то ничего хорошего бы не вышло, коэффициенты бы сели на границу этого ограничения. Но так как в statsmodels (да и в целом чаще всего) модель оценивается методом наименьших квадратов и без ограничений на параметры, это можно проигнорировать в нашем случае. Для прогнозирования особой разницы не будет, но часть статистических тестов работать перестанет из-за возможных смещений в распределениях оценок параметров.

3. (0.5 балла) Реализуйте метод forecast. По сути нужно просто спрогнозировать VAR и достать из прогноза только один ряд для нужной валюты.



In [ ]:
from statsmodels.tsa.api import VAR
import pandas as pd

class PartialForecaster:
    
    def __init__(self, data:pd.DataFrame, target_feature: str, p: int | None = None):
        """
        __init__ Initializing forecaster class

        Arguments:
            data {pd.DataFrame} -- Dataframe of all data
            target_feature {str} -- Target feature of dataframe. Other features in dataframe will be regressors
            p {int} -- Order of VAR model
        """
        self.is_pca_fitted = False
        self.is_var_fitted = False

        assert target_feature in data.columns

        self.data = data 
        self.target_feature = target_feature
        self.p = p

        
        
    def fit_pca(self, covered_variance=0.95) -> pd.DataFrame:
        """
        fit_pca _summary_

        Arguments:
            features {pd.DataFrame} -- Dataframe of features

        Keyword Arguments:
            covered_variance {float} -- Share of variance, covered by PCA from original data. (default: {0.95})

        Returns:
            pd.DataFrame -- Dataframe of principal components
        """


        # Your code
        
        self.is_pca_fitted = True

    def fit_var(self):

        """
         Fitting var on results of PCA
        """

        assert self.is_pca_fitted

        self.model = ... # Your code
        

        self.is_var_fitted = True


    def forecast(self, h: int = 12) -> pd.Series:
        """
        forecast Forecast the entire model from the end of training data h steps forward

        Keyword Arguments:
            h {int} -- Forecasting horizon (default: {1})

        Returns:
            pd.Series -- Forecast of target_feature
        """

        assert self.is_pca_fitted and self.is_var_fitted 

        # Your code

        forecast = ...

        assert type(forecast) is pd.Series


5. (1 балл) Отберите параметр p и посчитайте прогнозную ошибку такого метода для каждой валюты. 

    Предлагается следующая процедура.

    1. Выбираем некоторое стартовое окно. У нас 283 точки. 36 (12*3) последних точек выделим на тестовую часть, остальное на тренировочную.
    2. На тренировочной части выбираем p по информационному критерию. В классе выше это означает p = None

    Далее попробуем получить наиболее репрезентативную оценку ошибки для такого p. Мы будем сразу считать относительную ошибку в сравнении с наивной моделью, так как на котировках часто нельзя построить статистическую модель лучше наивной. 

    1. Прогнозируем полученной моделью на 12 шагов вперёд.
    2. Считаем абсолютную ошибку прогноза (вектор длины 12)
    3. Строим наивный прогноз
    4. Считаем абсолютную ошибку наивного прогноза
    5. Считаем отношение ошибки нашего прогноза к наивной ошибке, сохраняем этот вектор длины 12
    7. Увеличиваем тренировочную выборку на 6 наблюдений. Переоцениваем всю модель на новых данных, но уже при фиксированном p. В классе выше это будет p={некоторое число}

    Повторяем эту процедуру пока не закончатся данные. При текущих параметрах получится 4 итерации. Усредняем вектора ошибок по всем итерациям. Итого получаем усреднённый вектор длины 12. Параметры можете поменять по собственным соображениям, но поясните логику.


In [ ]:
def relative_score_partial(initial_window_size: int = 247, step: int = 5, horizon: int = 12) -> pd.Series:
    """
    cross_val_score Estimating MAE cross-val score on a fitted model. Needed to get more sustainable MAE estimation

    Arguments:
        initial_window_size {int} -- Initial size of expanding window
        step {int} -- Step size of expanding window
        horizon {int} -- Forecasting horizon of cross-validation score
        p {int} -- order of VAR model
    Returns:
        pd.Series -- Vector of absolute error by each horizon, averaged by several folds
    """

    # Your code

    score = ...

    assert type(score) is pd.Series

    return score

6. (0.5 балла) Постройте линейный график скоров. Можно строить любой другой график кроме линейного, если он будет более интерпретируем. Простор для творчества!

    1.  Каждая валюта представлена линией на графике
    2.  По оси абсцисс идёт горизонт прогнозирования (от 1 до 12)
    3.  По оси ординат отношение ошибки VAR-модели к ошибке наивной модели

Получилось ли по какой-то валюте стабильно предсказывать лучше наивной на всех горизонтах. По каким валютам получилось лучше, по каким хуже?

У нас получалось, что на сильно зарегулированных экономиках (например, Китай) прогнозы близки к наивным, так как курс валюты сильнее зависит от решений партии, а не внешней обстановки. Но в целом у вас может получиться и другой результат.

In [ ]:
# ༼ つ ◕_◕ ༽つ

# 3. Автокодировщик на стероидах

Можно построить альтернативный вариант.

1. Энкодер. Строим PCA на всех валютах (и доп фичах, если хотите) сразу, выделяем главные компоненты.
2. Декодер. PCA не обратим в стандартном понимании, нельзя аналитически получить из главных компонент обратно исходные ряды. Для каждой валюты обучаем регрессионную модель, предсказывающую курс валюты по главным компонентам. Если у нас 8 валют, здесь получится 8 моделей. Модели можно взять любые (линрег, бустинг, ...). Не мучайтесь с подбором гиперпараметров, бустинга из коробки хватит.
2. Строим VAR(p)-модель только на главных компонентах, отобрав p
4. Прогнозируем главные компоненты вперёд
5. Применяем модели-декодеры, чтобы получить из прогнозов главных компоненты прогнозы валют


1. (0.5 балла) Реализуйте метод fit_pca, который вычисляет главные компоненты от фичей и сохраняет их. Компоненты должны покрывать 95% дисперсии оригинальных данных. В этот раз нужно взять в вычисление все доступные ряды

2. (0.5 балла) Реализуйте метод fit_var. В нём необходимо склеить целевую переменную и полученные главные компоненты в один датасет и обучить на этом VAR(p)-модель.

3. (1 балл) Реализуйте метод fit_decoders. Необходимо оценить ряд регрессий, восстанавливающих валюты из главных компонент. Восстанавливать дополнительные переменные не нужно. Заморачиваться с отбором параметров этих моделей не нужно (только при большом желании). Бустинга из коробки должно хватить.

4. (0.5 балла) Реализуйте метод forecast. VAR прогнозируют главные компоненты, а модели-декодеры восстанавливают из этих прогнозов валюты. Функция должна возвращать датафрейм прогнозов всех валют на 12 шагов. По колонкам идут валюты, по строкам горизонты.

In [ ]:
from statsmodels.tsa.api import VAR
import pandas as pd

class EncoderDecoderForecaster:
    
    def __init__(self, data:pd.DataFrame, target_feature: str, p: int|None = None):
        """
        __init__ Initializing forecaster class

        Arguments:
            data {pd.DataFrame} -- Dataframe of all data
            target_feature {str} -- Target feature of dataframe. Other features in dataframe will be regressors
            p {int} -- Order of VAR model
        """
        self.is_pca_fitted = False
        self.is_var_fitted = False

        assert target_feature in data.columns

        self.data = data 
        self.target_feature = target_feature
        self.p = p

        self.decoders = []
        
        
    def fit_pca(self, covered_variance=0.95) -> pd.DataFrame:
        """
        fit_pca Fitting PCA on a bunch of currencies

        Arguments:
            features {pd.DataFrame} -- Dataframe of features

        Keyword Arguments:
            covered_variance {float} -- Share of variance, covered by PCA from original data. (default: {0.95})

        Returns:
            pd.DataFrame -- Dataframe of principal components
        """


        # Your code
        
        self.is_pca_fitted = True


    def fit_var(self):

        """
         Fitting var on results of PCA
        """

        assert self.is_pca_fitted

        self.model = ... # Your code
        

        self.is_var_fitted = True

    def fit_decoders(self):

        # Your code

        assert len(self.decoders) > 0
        self.is_decoder_fitted = True


    def forecast(self, h: int = 12) -> pd.Series:
        """
        forecast Forecast VAR and decode it's forecasts with decoder-models.

        Keyword Arguments:
            h {int} -- Forecasting horizon (default: {1})

        Returns:
            pd.DataFrame -- Forecasts of all currencies. Horizon by rows. Currencies by columns.
        """

        assert self.is_pca_fitted and self.is_var_fitted and self.is_decoder_fitted

        # Your code

        forecast = ...

        assert type(forecast) is pd.Series


5. (1 балл) Аналогично предыдущей части, подсчитайте относительную ошибку по всем валютам. Отличие будет только в том, что в этот раз класс EncoderDecoderForecaster возвращает прогнозы по всем валютам сразу. В данном случае итог будет сразу датафреймом. По колонкам будут валюты, по строкам горизонты. Параметр p отберите на тренировочной выборке и усредните относительную ошибку прогноза (усреднить датафреймы поэлементно) по всем окнам.

In [3]:
def relative_cross_val_score_encoder_decoder(self, data: pd.DataFrame, initial_window_size: int, step: int, p: int, horizon: int = 12) -> pd.Series:
    """
    cross_val_score Estimating MAE cross-val score on a fitted model. Needed to get more sustainable MAE estimation

    Arguments:
        initial_window_size {int} -- Initial size of expanding window
        step {int} -- Step size of expanding window
        horizon {int} -- Forecasting horizon of cross-validation score
        p {int} -- order of VAR model
    Returns:
        pd.DataFrame -- Frame of absolute error by each horizon and each currency, averaged by several folds
    """

    assert self.model is not None

    # Your code

    score = ...

    assert type(score) is pd.DataFrame

    return score

6. (0.5 балла) Постройте график относительных ошибок аналогично предыдущей части. Получилось ли добиться улучшения относительно предыдущей модели? Результат здесь не очевиден, метод может оказаться как лучше, так и хуже.

In [ ]:
# ༼ つ ◕_◕ ༽つ

# Расширения модели (2 балла)

Придумайте способ добавить в PCA (t-sne или любой другой кросс-секционный метод) зависимость от времени. Либо найдите метод понижения размерности специально для временных рядов. Кратко опишите суть и примените его для обоих классов выше вместо PCA. Получилось ли улучшить качество модели?

1. (1 балл) Ваше краткое описание методики и почему она подходит для рядов. Если есть статьи, можете приложить ссылки.

2. (1 балл) Реализация для обеих вариантов, подсчёт метрик и анализ результата.

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Рубрика "как вам домашка?" (0.1 балла)

Пройдите короткий опрос. Это действительно важно. https://forms.gle/w3sV453spERTbGvr7